In [5]:
import requests
from pandas.io.json import json_normalize
import os
import pandas as pd
import time
import math
from datetime import datetime
import numpy as np
import sqlalchemy 
import psycopg2
import smtplib

In [6]:
company = 'GAME_STOP'
keyword = '"GME"'
date_str = '2020-10-01'
date = datetime.strptime(date_str, '%Y-%m-%d')
date

datetime.datetime(2020, 10, 1, 0, 0)

In [7]:
class PushShiftPull():
    def _init_(keyword,last_comment,last_submmision,company):
        self.company = company
        self.keyword = keyword
        self.last_comment = last_comment
        self.last_submission = last_submission
    def create_directories(self):
        def check_create(directory):
            if not os.path.exists(directory):
                os.makedirs(directory)
        #Topline
        self.directory = os.path.join(os.getcwd(),'Reddit_Content',self.company,self.keyword)
        check_create(self.directory)
        #Comments
        self.directory_comments = os.path.join(self.directory,'comments')
        check_create(self.directory_comments)
        #Posts
        self.directory_posts = os.path.join(self.directory,'posts')
        check_create(self.directory_posts)
        print(self.directory)
    def downloadFromUrl_comment(self):
        last_comment_timestamp = int(self.last_comment.timestamp())
        #print(last_comment_timestamp)
        #url = "https://api.pushshift.io/reddit/comment/search?q={}&after={}&limit=1000&sort=desc&before=".format(self.keyword,self.comment_max)
        url = "https://api.pushshift.io/reddit/comment/search?q={}&limit=1000&subreddit=wallstreetbets&sort=desc&after={}&before=".format(self.keyword,last_comment_timestamp )
        print(url)
        count = 0
        previous_epoch = int(datetime.utcnow().timestamp())
        dfs = []
        while True:
            #new_url = url+str(previous_epoch)
            new_url = url.format(self.keyword,last_comment_timestamp)+str(previous_epoch)
            print(new_url)
            json = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
            try:
                json_data = json.json()
            except:
                print('failed!')
                break
            if 'data' not in json_data:
                break
            objects = json_data['data']
            if len(objects) == 0:
                break
            print('no break')
            for object in objects:
                previous_epoch = object['created_utc'] - 1
                count += 1
                try:
                    if count == 1:
                        self.comment_df = json_normalize(object)
                    else:
                        df = json_normalize(object)
                        self.comment_df = pd.concat([self.comment_df,df],axis = 0,ignore_index = True,sort=False)
                except Exception as err:
                    print(f"Couldn't print comment: https://www.reddit.com{object['permalink']}")
                    print(traceback.format_exc())
            print("Saved {} comments through {}".format(count, datetime.fromtimestamp(previous_epoch).strftime("%Y-%m-%d")))
        try:
            #self.comment_df = pd.concat(dfs,axis = 0,ignore_index = True,sort=False)
            self.comment_df['created_utc'] = pd.to_datetime(self.comment_df['created_utc'],unit='s')
            self.comment_df['type'] = 'comment'
            columns = ['author','author_fullname','body','created_utc','id','link_id','parent_id','permalink','score','subreddit','subreddit_id','subreddit_name_prefixed','subreddit_type','total_awards_received','type']
            self.comment_df = self.comment_df.reindex(columns = columns)
            #self.comment_df.to_csv(self.filename_comments,index = False)
        except:
            print('Not comments for ' + str(self.keyword))
    def downloadFromUrl_submission(self):
        #url = "https://api.pushshift.io/reddit/submission/search?q={}&after={}&limit=1000&sort=desc&author=&before=".format(self.keyword,self.submission_max)
        #print(url)
        last_sumbission = int(self.last_submission.timestamp())
        url = "https://api.pushshift.io/reddit/submission/search?q={}&limit=1000&subreddit=wallstreetbets&sort=desc&author=&after={}&before=".format(self.keyword,last_sumbission)
        count = 0
        previous_epoch = int(datetime.utcnow().timestamp())
        dfs = []
        while True:
            new_url = url.format(self.keyword,last_sumbission)+str(previous_epoch)
            json = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
            print('got json')
            try:
                json_data = json.json()
            except:
                print('failed!')
                break
            if 'data' not in json_data:
                break
            objects = json_data['data']
            if len(objects) == 0:
                break
            for object in objects:
                previous_epoch = object['created_utc'] - 1
                count += 1
                #if object['is_self']:
                #    if 'selftext' not in object:
                #        continue
                try:
                    if count == 1:
                        self.submission_df = json_normalize(object)
                    else:
                        df = json_normalize(object)
                        self.submission_df = pd.concat([self.submission_df,df],axis = 0,ignore_index = True,sort=False)
                except Exception as err:
                    print(f"Couldn't print post: {object['url']}")
                    print(traceback.format_exc())
            print("Saved {} submissions through {}".format(count, datetime.fromtimestamp(previous_epoch).strftime("%Y-%m-%d")))
        try:
            self.submission_df['created_utc'] = pd.to_datetime(self.submission_df['created_utc'],unit='s')
            self.submission_df['type'] = 'submission'
            columns = ['author','author_fullname','created_utc','full_link','id','is_meta','is_original_content','is_reddit_media_domain','is_robot_indexable','is_self','is_video','media_only','num_comments','num_crossposts','permalink','score','selftext','subreddit','subreddit_id','subreddit_subscribers','subreddit_type','thumbnail','title','url','crosspost_parent','promoted','media.type']
            self.submission_df = self.submission_df.reindex(columns = columns)
        except:
            print('No DF for Submissions!')
        #self.submission_df.to_csv(self.filename_submission,index = False)
    def create_files(self):
        #comments
        print('Saving Comments')
        cdate_min = str(np.min(self.comment_df['created_utc']))
        cdate_max = str(np.max(self.comment_df['created_utc']))
        comment_stuff = [self.company,self.keyword,'comments',cdate_min,cdate_max]
        comments_files = '-'.join(comment_stuff )
        self.comment_df.to_csv(os.path.join(self.directory_comments,comments_files),index = False)
        print('Saved Comments')
        #Posts
        pdate_min = str(np.min(self.submission_df['created_utc']))
        pdate_max = str(np.max(self.submission_df['created_utc']))
        submission_stuff = [self.company,self.keyword,'posts',pdate_min,pdate_max]
        submission_file = '-'.join(submission_stuff)
        self.submission_df.to_csv(os.path.join(self.directory_posts,submission_file),index = False)

In [8]:
def Run_PushShift(keyword,company,last_comment,last_submission):
    print(keyword)
    print(last_comment)
    print(last_submission)
    #Associate Variables
    Test = PushShiftPull()
    Test.company = company
    Test.keyword = keyword
    Test.last_comment = last_comment
    Test.last_submission = last_submission
    #Run Code
    Test.create_directories()
    Test.downloadFromUrl_comment()
    Test.downloadFromUrl_submission()
    Test.create_files()
    return(Test)

In [9]:
Avery = Run_PushShift(keyword,company,date,date)

"GME"
2020-10-01 00:00:00
2020-10-01 00:00:00
/Users/Avery/Desktop/WSB/Reddit_Content/GAME_STOP/"GME"
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611781603
no break


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Saved 100 comments through 2021-01-24
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611485732
no break
Saved 200 comments through 2021-01-24
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611481724
no break
Saved 300 comments through 2021-01-24
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611479351
no break
Saved 400 comments through 2021-01-24
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611476435
no break
Saved 500 comments through 2021-01-24
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611474352
no break
Saved 600 comments through 2021-01-24
https://api.pushshift.io/reddit/comment/search?

no break
Saved 4600 comments through 2021-01-23
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611431824
no break
Saved 4700 comments through 2021-01-23
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611430959
no break
Saved 4800 comments through 2021-01-23
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611430179
no break
Saved 4900 comments through 2021-01-23
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611429356
no break
Saved 5000 comments through 2021-01-23
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611428757
no break
Saved 5100 comments through 2021-01-23
https://api.pushshift.io/reddit/

no break
Saved 9100 comments through 2021-01-23
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611379849
no break
Saved 9200 comments through 2021-01-23
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611379197
no break
Saved 9300 comments through 2021-01-23
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611378486
no break
Saved 9400 comments through 2021-01-22
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611377506
no break
Saved 9500 comments through 2021-01-22
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611376715
no break
Saved 9600 comments through 2021-01-22
https://api.pushshift.io/reddit/

no break
Saved 13600 comments through 2021-01-22
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611352409
no break
Saved 13700 comments through 2021-01-22
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611352036
no break
Saved 13800 comments through 2021-01-22
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611351702
no break
Saved 13900 comments through 2021-01-22
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611351392
no break
Saved 14000 comments through 2021-01-22
https://api.pushshift.io/reddit/comment/search?q="GME"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611351062
no break
Saved 14100 comments through 2021-01-22
https://api.pushshift.io/r

KeyboardInterrupt: 